데이터 전처리

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../"))

from utils import *
from MLP import train_and_evaluate
from permutation import permutation

# Load dataset
region="gwangju"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

(40140, 22)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'IR', 'PM10', '지점', '위도', '경도',
       'diff'],
      dtype='object')
(9486, 107)
VV             0.031626
WD_yanan       0.021084
WS_yanan       0.021084
TA_yanan       0.105419
TD_yanan       0.274088
HM_yanan       0.685220
PS_yanan       0.031626
PR             0.063251
VV_tongliao    0.010542
WD_tongliao    0.010542
WS_tongliao    0.010542
TA_tongliao    0.137044
TD_tongliao    0.253004
HM_tongliao    0.242463
PS_tongliao    0.010542
PR_tongliao    0.010542
VV_qingdao     0.094877
WD_qingdao     0.115960
WS_qingdao     0.115960
TA_qingdao     0.094877
TD_qingdao     0.200295
HM_qingdao     0.790639
PS_qingdao     0.094877
PR_qingdao     0.094877
VV_chifeng     0.063251
WD_chifeng     0.021084
WS_chifeng     0.063251
TA_chifeng     0.115960
TD_chifeng     0.263546
HM_chifeng     0.179211
PS_chifeng     0.031626
PR_chifeng     0

MLP-Model - 한국만

In [2]:
removed_columns=['TM', 'PM10','WW','CT','diff']

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)



Cross-validated Mean Absolute Error: 16.28573932647705
251/251 [==============================] - 0s 556us/step - loss: 434.6461 - mae: 12.2565
Test Set Mean Absolute Error: 12.256548881530762


질적 분석

In [3]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_korea, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 8529.5048828125
251/251 [==============================] - 0s 511us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 0.7661982715108024
Feature: WS, Importance: -0.06670477916468372
Feature: PA, Importance: 124.62334365288844
Feature: PS, Importance: 122.36934168328135
Feature: TA, Importance: -0.9410938273385909
Feature: TD, Importance: 0.07361567191546783
Feature: HM, Importance: 42.33978043633706
Feature: PV, Importance: 0.04527115532073367
Feature: CA_TOT, Importance: 0.08759738028129505
Feature: CA_MID, Importance: 0.16172963247245206
Feature: VS, Importance: 276.886871629173
Feature: TS, Importance: 3.5418752090938144
Feature: IR, Importance: 0.005768859523232095
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 276.886871629173
Feature: PA, Importance: 124.62334365288844
Feature: PS, Importance: 122.3

MLP model - 중국 데이터 포함

In [4]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 18.932682228088378
60/60 [==============================] - 0s 631us/step - loss: 294.7962 - mae: 12.1814
Test Set Mean Absolute Error: 12.18136215209961


질적 분석

In [5]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features_china, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 15827.62109375
60/60 [==============================] - 0s 484us/step
Feature: STN, Importance: 0.0
Feature: WD, Importance: 0.00013071428147668485
Feature: WS, Importance: 9.257534766220488e-05
Feature: PA, Importance: 0.00011388941675249953
Feature: PS, Importance: 0.00011292468770989217
Feature: TA, Importance: 0.00010885996489378158
Feature: TD, Importance: 0.00010379192499385682
Feature: HM, Importance: 5.362604315450881e-05
Feature: PV, Importance: 0.00011329771659802645
Feature: CA_TOT, Importance: 8.641394924779888e-05
Feature: CA_MID, Importance: 7.082393640303052e-05
Feature: VS, Importance: 0.00012302861141506584
Feature: TS, Importance: 0.00013538356688513886
Feature: IR, Importance: 3.9862582707428376e-05
Feature: 지점, Importance: 0.0
Feature: 위도, Importance: 0.0
Feature: 경도, Importance: 0.0
Feature: STN_yanan, Importance: 0.0
Feature: IW, Importance: 0.0
Feature: IR_yanan, Importance: 0.0
Feature: IX, Importance: 0.0
Feature: VV, Imp

## MLP 한국만
Feature Importance 음수인 feature 제거한 버전

In [6]:
print(low_importance_features_korea)

['STN', 'WS', 'TA', '지점', '위도', '경도']


In [7]:
removed_columns=['TM', 'PM10','WW','CT','diff']
removed_columns+=low_importance_features_korea

# Features (X) and target (y)
X = korea_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = korea_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 16.098696517944337
251/251 [==============================] - 0s 581us/step - loss: 433.5833 - mae: 12.3964
Test Set Mean Absolute Error: 12.396443367004395


In [8]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 9431.369140625
251/251 [==============================] - 0s 688us/step
Feature: WD, Importance: 0.6367073426868955
Feature: PA, Importance: 168.13611349848088
Feature: PS, Importance: 94.72727757643061
Feature: TD, Importance: 0.027931802444982168
Feature: HM, Importance: 40.29571382454178
Feature: PV, Importance: -0.06273306426937779
Feature: CA_TOT, Importance: 0.08290289508004207
Feature: CA_MID, Importance: 0.21726145418851955
Feature: VS, Importance: 292.1495836211367
Feature: TS, Importance: 3.2588536278432003
Feature: IR, Importance: -0.037447868845083575

Top 10 Important Features (Sorted Descending by Importance Score):
Feature: VS, Importance: 292.1495836211367
Feature: PA, Importance: 168.13611349848088
Feature: PS, Importance: 94.72727757643061
Feature: HM, Importance: 40.29571382454178
Feature: TS, Importance: 3.2588536278432003
Feature: WD, Importance: 0.6367073426868955
Feature: CA_MID, Importance: 0.21726145418851955
Feature: CA_

## MLP 중국포함
Feature Importance 음수인 feature 제거한 버전

In [9]:
print(low_importance_features_china)

['STN', '지점', '위도', '경도', 'STN_yanan', 'IW', 'IR_yanan', 'IX', 'RH', 'LON', 'LAT', 'IW_tongliao', 'IR_tongliao', 'IX_tongliao', 'RH_tongliao', 'LON_tongliao', 'LAT_tongliao', 'IW_qingdao', 'IR_qingdao', 'IX_qingdao', 'RH_qingdao', 'LON_qingdao', 'LAT_qingdao', 'IW_chifeng', 'IR_chifeng', 'IX_chifeng', 'RH_chifeng', 'LON_chifeng', 'LAT_chifeng', 'IW_dalian', 'IR_dalian', 'IX_dalian', 'RH_dalian', 'LON_dalian', 'LAT_dalian']


In [10]:
removed_columns=['TM', 'PM10','WW','CT','STN_dalian','STN_tongliao','STN_qingdao','STN_chifeng','diff']
removed_columns+=low_importance_features_china

# Features (X) and target (y)
X = china_data.drop(columns=removed_columns)

# 결측값 처리하는 세가지 방안
#X.fillna(0, inplace=True)
X = X.fillna(X.mean())
#X.fillna(X.median(), inplace=True)

y = china_data['PM10']

# Train the model and get the necessary data
mean_mae, test_mae, trained_model, X_train_val, y_train_val, X_test, y_test = train_and_evaluate(
    X, y, test_size=0.2, n_splits=5, epochs=100, batch_size=64
)

Cross-validated Mean Absolute Error: 20.281172943115234
60/60 [==============================] - 0s 501us/step - loss: 297.5797 - mae: 12.3587
Test Set Mean Absolute Error: 12.358697891235352


In [11]:
# Evaluate the model and calculate permutation importance
mean_mae, low_importance_features, features_with_scores = permutation(
    trained_model, X_train_val, y_train_val, X_test, y_test
)

Model Evaluation - Mean Absolute Error: 10438.0849609375
60/60 [==============================] - 0s 467us/step
Feature: WD, Importance: 0.06331469585584273
Feature: WS, Importance: -0.010819298449678171
Feature: PA, Importance: 61.802654495034766
Feature: PS, Importance: 52.01063821956759
Feature: TA, Importance: 0.5494796836341266
Feature: TD, Importance: -0.35500503915336595
Feature: HM, Importance: 14.195932687196
Feature: PV, Importance: -0.09778229388466571
Feature: CA_TOT, Importance: 0.22754662423849367
Feature: CA_MID, Importance: 0.07834636351090012
Feature: VS, Importance: 139.05521181903515
Feature: TS, Importance: 1.118306570664572
Feature: IR, Importance: -0.003527330371980497
Feature: VV, Importance: 0.011434555355663178
Feature: WD_yanan, Importance: -0.006978772160800872
Feature: WS_yanan, Importance: 0.001612077548270463
Feature: TA_yanan, Importance: -0.011630450736629428
Feature: TD_yanan, Importance: 0.26540996603798706
Feature: HM_yanan, Importance: -1.72568084984